<a href="https://colab.research.google.com/github/valogonor/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [38]:
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data'
names = []
for i in range(16):
  names.append('A' + str(i+1))
df = pd.read_csv(url, names=names, na_values=['?'])
df.sample(5)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
257,b,20.00,0.000,u,g,d,v,0.500,f,f,0,f,g,144.0,0,-
508,a,30.67,12.000,u,g,c,v,2.000,t,t,1,f,g,220.0,19,+
159,b,34.08,0.080,y,p,m,bb,0.040,t,t,1,t,g,280.0,2000,+
146,b,23.25,1.500,u,g,q,v,2.375,t,t,3,t,g,0.0,582,+
408,b,16.00,3.125,u,g,w,v,0.085,f,t,1,f,g,0.0,6,-


In [12]:
df.shape # 690 is correct # of observations

(690, 16)

In [20]:
df.isna().sum()

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [25]:
df.dtypes # A11 & A15 are ints, but should be floats.

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

In [52]:
# Change A11 & A15 to floats
df['A11'] = df['A11'].astype(float)
df['A15'] = df['A15'].astype(float)
df.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11    float64
A12     object
A13     object
A14    float64
A15    float64
A16     object
dtype: object

In [53]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1.0,f,g,202.0,0.0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280.0,824.0,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,+


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [0]:
# TODO
df_approved = df[df.A16=='+']
df_rejected = df[df.A16=='-']

In [41]:
df_approved.sample(5)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
143,b,22.33,11.000,u,g,w,v,2.000,t,t,1,f,g,80.0,278,+
47,b,31.92,4.460,u,g,cc,h,6.040,t,t,3,f,g,311.0,300,+
585,b,73.42,17.750,u,g,ff,ff,0.000,t,f,0,t,g,0.0,0,+
63,a,20.42,0.835,u,g,q,v,1.585,t,t,1,f,g,0.0,0,+
133,b,36.25,5.000,u,g,c,bb,2.500,t,t,6,f,g,0.0,367,+


In [58]:
df_approved.A11 = df_approved.A11.astype(float)
df_approved.A15 = df_approved.A15.astype(float)
df_rejected.A11 = df_rejected.A11.astype(float)
df_rejected.A15 = df_rejected.A15.astype(float)
df_rejected.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
70,b,32.33,7.5,u,g,e,bb,1.585,t,f,0.0,t,s,420.0,0.0,-
71,b,34.83,4.0,u,g,d,bb,12.500,t,f,0.0,t,g,NaN,0.0,-
72,a,38.58,5.0,u,g,cc,v,13.500,t,f,0.0,t,g,980.0,0.0,-
73,b,44.25,0.5,u,g,m,v,10.750,t,f,0.0,f,s,400.0,0.0,-
74,b,44.83,7.0,y,p,c,v,1.625,f,f,0.0,f,g,160.0,2.0,-


In [49]:
from scipy import stats
stats.ttest_ind(df_approved.A2.dropna(), df_rejected.A2.dropna()) # The approved have higher A2 scores.

Ttest_indResult(statistic=4.2922156166315535, pvalue=2.027663707178064e-05)

In [50]:
stats.ttest_ind(df_approved.A3, df_rejected.A3) # The approved have higher A3 scores.

Ttest_indResult(statistic=5.52998337614816, pvalue=4.551680702308068e-08)

In [51]:
stats.ttest_ind(df_approved.A8, df_rejected.A8) # The approved have higher A8 scores.

Ttest_indResult(statistic=8.935819983773698, pvalue=3.6710537401601785e-18)

In [59]:
stats.ttest_ind(df_approved.A11, df_rejected.A11) # The approved have higher A11 scores.

Ttest_indResult(statistic=11.667004222431277, pvalue=7.957718568079967e-29)

In [61]:
stats.ttest_ind(df_approved.A14.dropna(), df_rejected.A14.dropna()) # The approved have lower A14 scores.

Ttest_indResult(statistic=-2.6358251986645476, pvalue=0.008586135473980122)

In [62]:
stats.ttest_ind(df_approved.A15, df_rejected.A15) # The approved have higher A15 scores.

Ttest_indResult(statistic=4.680216020964486, pvalue=3.4520256956287944e-06)

In [63]:
ct = pd.crosstab(df.A16, df.A1)
ct

A1,a,b
A16,,
+,98,206
-,112,262


In [69]:
stats.chisquare(ct) # Approved & rejected not much different in A1.

Power_divergenceResult(statistic=array([0.93333333, 6.7008547 ]), pvalue=array([0.33399826, 0.00963667]))

In [74]:
ct2 = pd.crosstab(df.A16, df.A4)
ct2

A4,l,u,y
A16,,,
+,2,256,45
-,0,263,118


In [75]:
stats.chisquare(ct2) # The rejected have way more y's in A4 than the approved do.

Power_divergenceResult(statistic=array([ 2.        ,  0.09441233, 32.69325153]), pvalue=array([1.57299207e-01, 7.58640991e-01, 1.07911231e-08]))

In [76]:
ct3 = pd.crosstab(df.A16, df.A5)
ct3

A5,g,gg,p
A16,,,
+,256,2,45
-,263,0,118


In [77]:
stats.chisquare(ct3) # The rejected have way more p's in A5 than the approved do.

Power_divergenceResult(statistic=array([ 0.09441233,  2.        , 32.69325153]), pvalue=array([7.58640991e-01, 1.57299207e-01, 1.07911231e-08]))

In [78]:
ct4 = pd.crosstab(df.A16, df.A6)
ct4

A6,aa,c,cc,d,e,ff,i,j,k,m,q,r,w,x
A16,,,,,,,,,,,,,,
+,19,62,29,7,14,7,14,3,14,16,51,2,33,32
-,35,75,12,23,11,46,45,7,37,22,27,1,31,6


In [79]:
stats.chisquare(ct4) # Significant differences are with aa, cc, d, ff, i, k, q, and x with A6.

Power_divergenceResult(statistic=array([ 4.74074074,  1.23357664,  7.04878049,  8.53333333,  0.36      ,
       28.69811321, 16.28813559,  1.6       , 10.37254902,  0.94736842,
        7.38461538,  0.33333333,  0.0625    , 17.78947368]), pvalue=array([2.94563856e-02, 2.66712592e-01, 7.93192353e-03, 3.48700489e-03,
       5.48506236e-01, 8.45861040e-08, 5.44035222e-05, 2.05903211e-01,
       1.27902833e-03, 3.30390049e-01, 6.57841361e-03, 5.63702862e-01,
       8.02587349e-01, 2.46744573e-05]))

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

The numerical attribute with the greatest difference between the approved and rejected is A11. Those who were approved for credit had significantly higher A11 scores than those who were rejected. The numerical attribute with the least difference between the approved and rejected is A14. The approved had lower A14 scores than the rejected, though the difference is still significant.

The approved and rejected are not much different in A1. The rejected have way more y's in A4 than the approved do.

The most challenging part of this sprint challenge was debugging as I worked on these problems.